In [2]:
streamlit_code = """
import streamlit as st
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

@st.cache_data
def load_resource_data():
    df = pd.read_csv("hospital_resources.csv")
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values("Date")
    return df

@st.cache_data
def load_arrival_data():
    df = pd.read_csv("patient_arrivals.csv")
    df["date"] = pd.to_datetime(df["date"]).dt.date
    return df

resources_df = load_resource_data()
arrivals_df = load_arrival_data()
latest = resources_df.iloc[-1].copy()

total_beds = 100
initial_resources = {
    "icu": int(latest["ICU_Available"]),
    "mri": int(latest["MRI_In_Use"]),
    "general": total_beds - int(latest["Bed_Occupancy_Rate"]),
    "ventilators": int(latest["Ventilators_In_Use"])
}
if "resource_state" not in st.session_state:
    st.session_state.resource_state = initial_resources.copy()
if "queue" not in st.session_state:
    st.session_state.queue = []

if st.button("🔁 Reset Queue & Resources"):
    st.session_state.queue = []
    st.session_state.resource_state = initial_resources.copy()
    st.success("System reset successfully.")

st.title("🏥 Smart Hospital Resource Management Dashboard")

# Resource Display
st.subheader("📊 Current Hospital Resource Status")
res = st.session_state.resource_state
st.metric("ICU Beds", f"{res['icu']}")
st.metric("General Beds", f"{res['general']}")
st.metric("MRI Usage", f"{res['mri']}")
st.metric("Ventilators", f"{res['ventilators']}")

# Patient Registration
st.header("📝 Register New Patient")
with st.form("patient_form"):
    name = st.text_input("Name")
    age = st.number_input("Age", 0, 120, 30)
    problem = st.text_area("Describe the problem")
    urgency = st.selectbox("Urgency Level", ["Low", "Medium", "High"])
    if urgency == "Medium":
        resource_choice = st.radio("Allocate to:", ["ICU", "General", "Ventilator"])
    else:
        resource_choice = None
    submit = st.form_submit_button("Submit")

# Queue Handling
if submit:
    msg = ""
    if urgency == "High":
        if res["icu"] > 0:
            res["icu"] -= 1
            msg += "✅ ICU allocated. "
        else:
            msg += "⚠️ ICU not available. "

        if res["mri"] < 10:
            res["mri"] += 1
            msg += "MRI used. "
        else:
            msg += "⚠️ MRI not available. "

        if res["ventilators"] < 10:
            res["ventilators"] += 1
            msg += "Ventilator allocated. "
        else:
            msg += "⚠️ Ventilator not available. "

    elif urgency == "Medium":
        if resource_choice == "ICU" and res["icu"] > 0:
            res["icu"] -= 1
            msg = f"✅ ICU bed allocated to {name}"
        elif resource_choice == "General" and res["general"] > 0:
            res["general"] -= 1
            msg = f"✅ General ward bed allocated to {name}"
        elif resource_choice == "Ventilator" and res["ventilators"] < 10:
            res["ventilators"] += 1
            msg = f"✅ Ventilator allocated to {name}"
        else:
            msg = f"⚠️ {resource_choice} not available"

    else:  # Low
        if res["general"] > 0:
            res["general"] -= 1
            msg = f"✅ General ward bed allocated to {name}"
        else:
            msg = f"⚠️ No general ward beds available"

    patient = {
        "name": name,
        "age": age,
        "problem": problem,
        "urgency": urgency,
        "timestamp": datetime.now()
    }
    st.session_state.queue.append(patient)

    # Sort queue by urgency: High > Medium > Low
    urgency_rank = {"High": 0, "Medium": 1, "Low": 2}
    st.session_state.queue.sort(key=lambda x: urgency_rank[x["urgency"]])

    st.success(f"{name} added to the queue.")
    st.info(msg)

# Patient Queue
st.header("📋 Current Patient Queue")
avg_time = {"High": 10, "Medium": 20, "Low": 30}

# Always sort before displaying
urgency_rank = {"High": 0, "Medium": 1, "Low": 2}
st.session_state.queue.sort(key=lambda x: urgency_rank[x["urgency"]])

if st.session_state.queue:
    for i, patient in enumerate(st.session_state.queue):
        wait = 0 if i == 0 else sum([avg_time[p["urgency"]] for p in st.session_state.queue[:i]])
        eta = datetime.now() + timedelta(minutes=wait)
        col1, col2 = st.columns([4, 1])
        with col1:
            st.markdown(
                f"**{i+1}. {patient['name']} (Age {patient['age']})**\\n"
                f"- Problem: {patient['problem']}\\n"
                f"- Urgency: `{patient['urgency']}`\\n"
                f"- Estimated Admission: ⏰ **{eta.strftime('%H:%M:%S')}** ({wait} mins)"
            )
        with col2:
            if st.button(f"✅ Complete {patient['name']}", key=f"done_{i}"):
                st.session_state.queue.pop(i)
                st.rerun()
else:
    st.info("No patients in queue.")

# Urgency Chart
st.subheader("👥 Queue Urgency Mix")
if st.session_state.queue:
    urgency_df = pd.DataFrame(st.session_state.queue)
    urgency_chart = urgency_df["urgency"].value_counts().reset_index()
    urgency_chart.columns = ["Urgency", "Count"]
    st.bar_chart(urgency_chart.set_index("Urgency"))

# Forecasting
st.header("📈 Forecasting: Beds & Occupancy")

# ARIMA Forecast
st.subheader("🔮 ARIMA: Bed Occupancy Forecast (7 Days)")
ts = resources_df.set_index("Date")["Bed_Occupancy_Rate"].resample("D").mean().interpolate()
arima_model = ARIMA(ts, order=(3, 1, 2)).fit()
arima_forecast = arima_model.forecast(steps=7)
st.line_chart(arima_forecast)

# XGBoost Forecasts
st.subheader("🤖 XGBoost: Predicted Beds Needed")
arrivals_df["dayofweek"] = pd.to_datetime(arrivals_df["date"]).dt.dayofweek
arrivals_df["trend"] = np.arange(len(arrivals_df))
X = arrivals_df[["dayofweek", "trend"]]
y = arrivals_df["patients_queued"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
model = XGBRegressor(n_estimators=100)
model.fit(X_train, y_train)

# Monthly Forecast
future_30 = pd.DataFrame({
    "dayofweek": [(datetime.today() + timedelta(days=i)).weekday() for i in range(30)],
    "trend": np.arange(len(arrivals_df), len(arrivals_df) + 30)
})
future_30["predicted_beds"] = np.round(model.predict(future_30)).astype(int)
future_30["date"] = [datetime.today().date() + timedelta(days=i) for i in range(30)]

st.markdown("### 📅 Monthly Forecast (Next 30 Days)")
st.line_chart(future_30.set_index("date")["predicted_beds"])

# Yearly Forecast
future_365 = pd.DataFrame({
    "dayofweek": [(datetime.today() + timedelta(days=i)).weekday() for i in range(365)],
    "trend": np.arange(len(arrivals_df), len(arrivals_df) + 365)
})
future_365["predicted_beds"] = np.round(model.predict(future_365)).astype(int)
future_365["date"] = [datetime.today().date() + timedelta(days=i) for i in range(365)]
future_365["month"] = pd.to_datetime(future_365["date"]).dt.to_period("M")
monthly_avg = future_365.groupby("month")["predicted_beds"].mean().round().astype(int).reset_index()

st.markdown("### 📆 Yearly Forecast (Avg. Monthly Beds)")
st.line_chart(monthly_avg.set_index("month"))
"""
with open("app.py", "w", encoding="utf-8") as f:
    f.write(streamlit_code)

print("✅ Updated app.py has been saved. Run it using: !streamlit run app.py")


✅ Updated app.py has been saved. Run it using: !streamlit run app.py
